- 강수량 / mm
- 강수형태 / 비옴 : 1, 비안옴 : 0
- 강수확률 / % step : 10
- 기온 / 'C step : 1
- 습도 / % step : 5
- 풍속 / ms step 0.1
- 하늘 상태 / 맑음(1), 구름많음(3), 흐림(4)

- 강수량 / mm step : 1 -> 그냥 int
- 강수형태 / 비옴 : 1, 비안옴 : 0 -> 그냥 int
- 강수확률 / % step : 10 -> 그냥 int
- 기온 / 'C step : 1 -> 그냥 int
- 습도 / % step : 5 -> 그냥 int
- 풍속 / ms step 0.1 -> X 10 하고 int
- 하늘 상태 / 맑음(1), 구름많음(3), 흐림(4) -> 그냥 int

In [35]:
import numpy as np
import os

In [3]:
filePaths = {
	fileName.split('.')[0] : f"datasets/{fileName}" for fileName in os.listdir('datasets')
}

In [6]:
for path in filePaths.values():
	with open(path) as f:
		print(f"{path} : {len(f.readlines())}")

datasets/강수량.csv : 573831
datasets/강수형태.csv : 573831
datasets/강수확률.csv : 573831
datasets/기온.csv : 572304
datasets/습도.csv : 573831
datasets/풍속.csv : 573831
datasets/하늘상태.csv : 573831


기온에 missing data

In [53]:
rawData_SkyStatus = []
with open(filePaths['하늘상태']) as f:
	rawData_SkyStatus = list(map(lambda x : x.rstrip().lstrip().split(','), f.readlines()))

In [54]:
rawData_SkyStatus[:5]

[['format: day',
  'hour',
  'forecast',
  'value  location:55_123 Start : 20210629'],
 ['29', '1700', '+6', '4.000000'],
 ['29', '1700', '+7', '4.000000'],
 ['29', '1700', '+8', '3.000000'],
 ['29', '1700', '+9', '3.000000']]

In [77]:
data_SkyStatus = []
lastStart = 1
for i, objs in enumerate(rawData_SkyStatus[:-1]):
	if objs[0][:5] == "Start":
		data_SkyStatus.append(list(map(lambda x : list(map(lambda k : int(float(k)), x)), rawData_SkyStatus[lastStart:i])))
		lastStart = i+1
del data_SkyStatus[0]
# 0번째 데이터는 개수가 안 맞음

data_SkyStatus = np.array(data_SkyStatus, dtype=np.uint8)

In [78]:
data_SkyStatus

array([[[ 30, 200,   6,   3],
        [ 30, 200,   7,   3],
        [ 30, 200,   8,   3],
        ...,
        [ 30, 252,  62,   4],
        [ 30, 252,  63,   4],
        [ 30, 252,  64,   4]],

       [[  1, 200,   6,   1],
        [  1, 200,   7,   1],
        [  1, 200,   8,   1],
        ...,
        [  1, 252,  62,   1],
        [  1, 252,  63,   1],
        [  1, 252,  64,   1]],

       [[  2, 200,   6,   3],
        [  2, 200,   7,   3],
        [  2, 200,   8,   3],
        ...,
        [  2, 252,  62,   1],
        [  2, 252,  63,   3],
        [  2, 252,  64,   4]],

       ...,

       [[ 27, 200,   6,   4],
        [ 27, 200,   7,   4],
        [ 27, 200,   8,   4],
        ...,
        [ 27, 252,  62,   3],
        [ 27, 252,  63,   3],
        [ 27, 252,  64,   1]],

       [[ 28, 200,   6,   1],
        [ 28, 200,   7,   1],
        [ 28, 200,   8,   1],
        ...,
        [ 28, 252,  62,   1],
        [ 28, 252,  63,   1],
        [ 28, 252,  64,   1]],

       [[ 29

In [79]:
data_SkyStatus.shape

(1126, 508, 4)

In [81]:
def loadData(filePath : str, convertFunction):
	rawData = []
	with open(filePath) as f:
		rawData = list(map(lambda x : x.rstrip().lstrip().split(','), f.readlines()))

	data = []
	lastStart = 1
	for i, objs in enumerate(rawData[:-1]):
		if objs[0][:5] == "Start":
			data.append(list(map(lambda x : list(map(convertFunction, x)), rawData[lastStart:i])))
			lastStart = i+1
	del data[0]
	# 0번째 데이터는 개수가 안 맞음

	return np.array(data, dtype=np.uint8)

어떤 기온 데이터가 빠진걸까?

In [88]:
tmp = []
with open(filePaths['기온']) as f:
	tmp = list(map(lambda x : x.rstrip().lstrip().split(','), f.readlines()))
tempdate = []
for i, objs in enumerate(tmp[:-1]):
	if objs[0][:5] == "Start":
		tempdate.append(objs[0][8:])

In [89]:
tmp = []
with open(filePaths['강수량']) as f:
	tmp = list(map(lambda x : x.rstrip().lstrip().split(','), f.readlines()))
raindate = []
for i, objs in enumerate(tmp[:-1]):
	if objs[0][:5] == "Start":
		raindate.append(objs[0][8:])

In [91]:
set(raindate) - set(tempdate)

{' 20220116', ' 20220202', ' 20220203'}

이제 모든 데이터에서 저 날짜 데이터를 빼면 된다.
- 숫자를 고치자

그러나 저 날짜는 데이터를 나누는 분기점이다. 따라서 +1일을 하고 그 날짜일때 전 데이터를 지워준다면 될 것이다.

온도 데이터에는 이미 저 날짜들이 제외된 상태이니 arg를 추가해 조정하자

또한, 데이터의 value만 중요하므로 나머지 데이터를 버려주자

In [111]:
def loadData(filePath : str, convertFunction = lambda x : int(float(x)), removeDate : bool = True):
	rawData = []
	with open(filePath) as f:
		rawData = list(map(lambda x : x.rstrip().lstrip().split(','), f.readlines()))

	data = []
	lastStart = 1
	for i, objs in enumerate(rawData[:-1]):
		if objs[0][:5] == "Start":
			if removeDate and (objs[0][8:] in ['20220117', '20220203', '20220204']):
				lastStart = i+1
			else:
				data.append(list(map(lambda x : convertFunction(x[3]), rawData[lastStart:i])))
				lastStart = i+1
	del data[0]
	# 0번째 데이터는 개수가 안 맞음

	return np.array(data, dtype=np.int8)

- 풍속 / ms step 0.1 -> X 10 하고 int 를 생각하자

In [112]:
for key, value in filePaths.items():
	if key == '풍속':
		func = lambda x : int(float(x) * 10)
	else:
		func = lambda x : int(float(x))
	print(f"{key} {loadData(value, func, False if key == '기온' else True).shape}")

강수량 (1123, 508)
강수형태 (1123, 508)
강수확률 (1123, 508)
기온 (1123, 508)
습도 (1123, 508)
풍속 (1123, 508)
하늘상태 (1123, 508)
